In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tqdm import tqdm
from torchvision import models, transforms
from torchvision.models import VGG16_Weights
from torchvision import models, transforms
from torchvision.models import VGG16_Weights
class CSVDataset(Dataset):
    def __init__(self, csv_file, img_dir, class_to_idx, transform=None):
        self.df = pd.read_csv(csv_file, low_memory=False)
        self.img_dir = Path(img_dir)
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        raw_rel = str(row['rel_path'])
        rel = raw_rel.lstrip('/\\') 

        rel_pt = Path(rel).with_suffix('.pt')

        img_path = (self.img_dir / rel_pt)

        if not img_path.exists():
            fallback = self.img_dir / rel_pt.name
            if fallback.exists():
                img_path = fallback
            else:
                raise FileNotFoundError(
                    f"Could not find .pt for CSV entry '{raw_rel}'.\n"
                    f"Tried:\n  {str(self.img_dir / rel_pt)}\n  {str(fallback)}\n"
                    f"Make sure files are named exactly and rel_path entries don't start with a leading slash."
                )

        try:
            t = torch.load(str(img_path), map_location='cpu')
        except Exception as e:
            raise RuntimeError(f"Error loading tensor from {img_path}: {e}")

        if not isinstance(t, torch.Tensor):
            raise TypeError(f"Expected torch.Tensor loaded from {img_path}, got {type(t)}")

        if t.ndim == 4 and t.shape[0] == 1:
            t = t[0]

        if t.ndim == 2:
            t = t.unsqueeze(0)

        if t.ndim != 3:
            raise ValueError(f"Loaded tensor from {img_path} has unexpected shape {t.shape}; expected (C,H,W).")

        if not torch.is_floating_point(t):
            t = t.float()
            if float(t.max()) > 1.0:
                t = t / 255.0

        if self.transform is not None:
            t = self.transform(t)

        label = self.class_to_idx[row['true_class']]
        return t, label, str(rel_pt)


def _load_checkpoint_weights(model, checkpoint_path, map_location='cpu', strict=True, verbose=True):
    ckpt = torch.load(checkpoint_path, map_location=map_location)

    state = None
    if isinstance(ckpt, dict):
        for candidate in ('model_state', 'model_state_dict', 'state_dict', 'state', 'net'):
            if candidate in ckpt:
                state = ckpt[candidate]
                break
        if state is None:
            if all(isinstance(v, (torch.Tensor, type(None))) or hasattr(v, 'shape') for v in ckpt.values()):
                state = ckpt
            else:
                nested = None
                for v in ckpt.values():
                    if isinstance(v, dict):
                        if nested is None or len(v) > len(nested):
                            nested = v
                state = nested if nested is not None else ckpt
    else:
        state = ckpt

    if not isinstance(state, dict):
        raise ValueError(f"Checkpoint {checkpoint_path} does not contain a state-dict (found {type(state)})")

    keys = list(state.keys())
    prefix_to_strip = None
    for p in ('module.', 'model.'):
        cnt = sum(1 for k in keys if k.startswith(p))
        if cnt >= max(1, len(keys) * 0.5):
            prefix_to_strip = p
            break
    if prefix_to_strip:
        state = { (k[len(prefix_to_strip):] if k.startswith(prefix_to_strip) else k): v
                  for k, v in state.items() }

    diagnostics = {'loaded_strict': False, 'missing_keys': None, 'unexpected_keys': None}

    try:
        model.load_state_dict(state, strict=strict)
        diagnostics['loaded_strict'] = strict
        if verbose:
            print(f"Loaded checkpoint '{checkpoint_path}' (strict={strict}).")
        return model, diagnostics
    except Exception as e:
        if verbose:
            print(f"Warning: strict load failed ({e}). Trying strict=False ...")
        res = model.load_state_dict(state, strict=False)
        diagnostics['missing_keys'] = getattr(res, 'missing_keys', None)
        diagnostics['unexpected_keys'] = getattr(res, 'unexpected_keys', None)
        diagnostics['loaded_strict'] = False
        if verbose:
            print("Loaded checkpoint with strict=False.")
            print("Missing keys:", diagnostics['missing_keys'])
            print("Unexpected keys:", diagnostics['unexpected_keys'])
        return model, diagnostics


def evaluate_model(
    model,
    checkpoint_path,
    detect_csv,
    image_dir,
    class_to_idx,
    device=None,
    batch_size=32,
    mean=(0.4812775254249573, 0.4674863815307617, 0.4093940854072571),
    std=(0.19709135591983795, 0.1933959424495697, 0.19051066040992737),
    out_csv_path=None,
    verbose=True
):
    if device is None:
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    else:
        device = torch.device(device)

    detect_transform = transforms.Normalize(mean, std)

    dataset = CSVDataset(detect_csv, image_dir, class_to_idx, transform=detect_transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False,
                            pin_memory=(device.type == 'cuda'))

    model = model.to(device)
    _load_checkpoint_weights(model, checkpoint_path, map_location=device)
    model.eval()

    all_preds, all_labels, all_rels = [], [], []

    with torch.no_grad():
        for inputs, labels, rels in tqdm(dataloader, desc="Evaluating", disable=not verbose):
            inputs = inputs.to(device)
            outputs = model(inputs)
            if outputs.dim() == 1:
                preds = (outputs > 0.5).long()
            else:
                _, preds = torch.max(outputs, dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.tolist())
            all_rels.extend(rels)

    idx_to_class = {v: k for k, v in class_to_idx.items()}
    classes_sorted = [idx_to_class[i] for i in sorted(idx_to_class.keys())]

    report = classification_report(all_labels, all_preds, target_names=classes_sorted, zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    if verbose:
        print("\n=== Classification Report ===")
        print(report)
        print("=== Confusion Matrix ===")
        print(cm)
        print(f"\nMacro F1: {f1:.4f}")

    out_df = pd.DataFrame({
        'rel_path': all_rels,
        'true_idx': all_labels,
        'pred_idx': all_preds,
    })
    out_df['true_class'] = out_df['true_idx'].map(idx_to_class)
    out_df['pred_class'] = out_df['pred_idx'].map(idx_to_class)

    if out_csv_path:
        out_csv_path = Path(out_csv_path)
        out_csv_path.parent.mkdir(parents=True, exist_ok=True)
        out_df.to_csv(out_csv_path, index=False)
        if verbose:
            print(f"Saved detailed results to {out_csv_path}")

    return {
        'report_text': report,
        'confusion_matrix': cm,
        'macro_f1': f1,
        'per_image_df': out_df
    }


Matplotlib is building the font cache; this may take a moment.


In [2]:
DETECT_CSV = './Evaluate/common_success_details.csv'

class_to_idx = {c: i for i, c in enumerate(['bird', 'cat', 'fish', 'frog', 'monkey', 'moth', 'spider'])}

print (class_to_idx)

def image_dir(size):
    return Path(f'../02-AdvGenerate/output/{size}x{size}/fgsm_0.03/{size}/')

attack_labels = ['fgsm', 'bim', 'pgd', 'df']

{'bird': 0, 'cat': 1, 'fish': 2, 'frog': 3, 'monkey': 4, 'moth': 5, 'spider': 6}


In [3]:
from torchvision.models import VGG16_BN_Weights

model = models.vgg16_bn(weights=VGG16_BN_Weights.IMAGENET1K_V1)
model.avgpool = nn.AdaptiveAvgPool2d((7,7))
model.classifier[6] = nn.Linear(4096, 7)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to D:\Users\a1901443/.cache\torch\hub\checkpoints\vgg16_bn-6c64b313.pth
100%|███████████████████████████████████████████████████████████████████████████████| 528M/528M [00:12<00:00, 43.6MB/s]


In [4]:
IMG_SIZE = 32

for attack_label in attack_labels:
    print(f'Type: {attack_label} \n')
    metrics = evaluate_model(
        model=model,
        checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
        detect_csv=DETECT_CSV,
        image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
        class_to_idx=class_to_idx,
        batch_size=32,        
        out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
        verbose=True
    )

Type: fgsm 

Loaded checkpoint '../01-CleanModel/Models/32x32/best_min_acc_vgg16_32x32_Model-2.pth' (strict=True).


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 21/21 [00:23<00:00,  1.11s/it]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 28  6 31  7  6  9]
 [13  0  5 27  7  6 16]
 [ 3 10  0 28  4  7  4]
 [13 29 26  0 35 10 41]
 [ 8 26  6 48  0  5 50]
 [ 1  4 14 15  6  0 13]
 [ 7 13  1 37 17  7  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_32x32-fgsm.csv
Type: bim 

Loaded checkpoint '../01-CleanModel/Models/32x32/best_min_acc_vgg16_32x

Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 21/21 [00:33<00:00,  1.58s/it]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 33  6 22  7  6 13]
 [13  0  8 22 12  5 14]
 [10 11  0 14  4  7 10]
 [19 40 27  0 31  8 29]
 [13 25  7 47  0  7 44]
 [ 6  9 16 10  4  0  8]
 [ 7 14  1 35 14 11  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_32x32-bim.csv
Type: pgd 

Loaded checkpoint '../01-CleanModel/Models/32x32/best_min_acc_vgg16_32x3

Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 21/21 [00:34<00:00,  1.64s/it]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 39  5 19  6  9  9]
 [15  0  6 19 13  6 15]
 [ 5  9  0 23  5  7  7]
 [15 45 28  0 28  9 29]
 [16 23  3 50  0  6 45]
 [ 9  9 17  6  4  0  8]
 [10 10  5 34 14  9  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_32x32-pgd.csv
Type: df 

Loaded checkpoint '../01-CleanModel/Models/32x32/best_min_acc_vgg16_32x32

Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 21/21 [00:33<00:00,  1.60s/it]


=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.02      0.01      0.01        87
         cat       0.00      0.00      0.00        74
        fish       0.00      0.00      0.00        56
        frog       0.00      0.00      0.00       154
      monkey       0.00      0.00      0.00       143
        moth       0.00      0.00      0.00        53
      spider       0.00      0.00      0.00        82

    accuracy                           0.00       649
   macro avg       0.00      0.00      0.00       649
weighted avg       0.00      0.00      0.00       649

=== Confusion Matrix ===
[[ 1 48  8 16  4  3  7]
 [16  0  5 16 20  3 14]
 [11  5  0 34  2  3  1]
 [13 38 34  0 29  6 34]
 [13 40  1 42  0  2 45]
 [ 8  2 19  8  4  0 12]
 [ 4 12  0 37 17 12  0]]

Macro F1: 0.0019
Saved detailed results to Evaluate\raw\miss-detection_results_32x32-df.csv


In [5]:
IMG_SIZE = 64

for attack_label in attack_labels:
    print(f'Type: {attack_label} \n')
    metrics = evaluate_model(
        model=model,
        checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
        detect_csv=DETECT_CSV,
        image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
        class_to_idx=class_to_idx,
        batch_size=2,        
        out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
        verbose=True
    )

Type: fgsm 

Loaded checkpoint '../01-CleanModel/Models/64x64/best_min_acc_vgg16_64x64_Model-2.pth' (strict=True).


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [00:55<00:00,  5.87it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 32  7 25 15  2  6]
 [21  0  6 21 12  5  9]
 [ 1 11  0 33  4  3  4]
 [13 23 44  0 32  7 35]
 [10 35  1 30  0  7 60]
 [ 0  3 10 15  4  0 21]
 [ 6  6  3 35 22 10  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_64x64-fgsm.csv
Type: bim 

Loaded checkpoint '../01-CleanModel/Models/64x64/best_min_acc_vgg16_64x

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [00:59<00:00,  5.48it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 40  9 24  8  2  4]
 [24  0  4 16 13  6 11]
 [ 4 13  0 33  2  4  0]
 [16 25 36  0 37  5 35]
 [ 9 35  2 32  0 11 54]
 [ 1  3  8 16  3  0 22]
 [ 8  9  1 31 20 13  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_64x64-bim.csv
Type: pgd 

Loaded checkpoint '../01-CleanModel/Models/64x64/best_min_acc_vgg16_64x6

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [00:47<00:00,  6.87it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 50  8 17 10  1  1]
 [31  0  2 11 20  2  8]
 [ 3 18  0 31  1  3  0]
 [16 29 48  0 29  5 27]
 [11 31  6 30  0 11 54]
 [ 3  4  8 13  4  0 21]
 [ 6 10  2 33 19 12  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_64x64-pgd.csv
Type: df 

Loaded checkpoint '../01-CleanModel/Models/64x64/best_min_acc_vgg16_64x64

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [00:51<00:00,  6.37it/s]


=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 49  7 18 10  2  1]
 [28  0  3  7 24  2 10]
 [ 6 14  0 33  0  2  1]
 [21 15 44  0 36  6 32]
 [14 29  6 26  0  9 59]
 [ 5  4 12  9  4  0 19]
 [ 6  7  0 26 24 19  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_64x64-df.csv


In [6]:
IMG_SIZE = 128

for attack_label in attack_labels:
    print(f'Type: {attack_label} \n')
    metrics = evaluate_model(
        model=model,
        checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
        detect_csv=DETECT_CSV,
        image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
        class_to_idx=class_to_idx,
        batch_size=2,        
        out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
        verbose=True
    )

Type: fgsm 

Loaded checkpoint '../01-CleanModel/Models/128x128/best_min_acc_vgg16_128x128_Model-2.pth' (strict=True).


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [01:31<00:00,  3.56it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 61  1 15  7  1  2]
 [28  0  5 10 26  0  5]
 [ 0 20  0 29  5  0  2]
 [10 15 13  0 50  3 63]
 [ 9 60  5 19  0  2 48]
 [ 5  7  6  5  4  0 26]
 [ 3 11  2 23 31 12  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_128x128-fgsm.csv
Type: bim 

Loaded checkpoint '../01-CleanModel/Models/128x128/best_min_acc_vgg16

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [01:33<00:00,  3.46it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 60  2 14  8  1  2]
 [28  0  6  6 30  0  4]
 [ 1  8  0 43  3  0  1]
 [ 5 20 14  0 42  8 65]
 [ 9 59  2 18  0  7 48]
 [ 4  9  5  8  6  0 21]
 [ 4  6  3 27 27 15  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_128x128-bim.csv
Type: pgd 

Loaded checkpoint '../01-CleanModel/Models/128x128/best_min_acc_vgg16_

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [01:47<00:00,  3.04it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 60  3 16  7  0  1]
 [34  0  3  6 27  0  4]
 [ 4 10  0 38  3  0  1]
 [10 24 17  0 43 10 50]
 [ 5 64  7 23  0  6 38]
 [ 4  9  7  4  4  0 25]
 [ 7  4  2 30 23 16  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_128x128-pgd.csv
Type: df 

Loaded checkpoint '../01-CleanModel/Models/128x128/best_min_acc_vgg16_1

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [01:38<00:00,  3.31it/s]


=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 57  7 11  9  1  2]
 [22  0  2  8 34  1  7]
 [ 3  9  0 38  3  1  2]
 [10 26 47  0 34  3 34]
 [12 47  2 23  0  4 55]
 [ 9  9 12  5  5  0 13]
 [ 8  6  3 26 28 11  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_128x128-df.csv


In [7]:
IMG_SIZE = 256

for attack_label in attack_labels:
    print(f'Type: {attack_label} \n')
    metrics = evaluate_model(
        model=model,
        checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
        detect_csv=DETECT_CSV,
        image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
        class_to_idx=class_to_idx,
        batch_size=2,        
        out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
        verbose=True
    )

Type: fgsm 

Loaded checkpoint '../01-CleanModel/Models/256x256/best_min_acc_vgg16_256x256_Model-2.pth' (strict=True).


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [02:06<00:00,  2.57it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 33  4 11 19  1 19]
 [12  0  3  4 53  0  2]
 [ 5 20  0 30  1  0  0]
 [10 13 21  0 18  1 91]
 [14 61  4 15  0  0 49]
 [10 11  2  7  8  0 15]
 [ 6  9  7 30 15 15  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_256x256-fgsm.csv
Type: bim 

Loaded checkpoint '../01-CleanModel/Models/256x256/best_min_acc_vgg16

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [02:03<00:00,  2.62it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[  0  33   4  13  16   1  20]
 [  9   0   4   4  52   1   4]
 [  2   5   0  46   0   1   2]
 [ 10   5  24   0  10   4 101]
 [ 13  53   4  21   0   0  52]
 [ 12   7   2  11   5   0  16]
 [  6   7   5  33  11  20   0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_256x256-bim.csv
Type: pgd 

Loaded checkpoint '..

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [02:22<00:00,  2.27it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 47  4  9 17  0 10]
 [14  0  2  0 53  2  3]
 [ 7 18  0 28  0  0  3]
 [15 17 18  0 13  3 88]
 [19 60  2 20  0  2 40]
 [10 11  1  7 13  0 11]
 [ 7  7  0 28 25 15  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_256x256-pgd.csv
Type: df 

Loaded checkpoint '../01-CleanModel/Models/256x256/best_min_acc_vgg16_2

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [02:23<00:00,  2.26it/s]


=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 54  7 14  9  0  3]
 [24  0  3  9 32  2  4]
 [ 5  6  0 39  0  3  3]
 [ 8 11 84  0 11  4 36]
 [20 42  5 22  0  3 51]
 [ 8 11  9 10  5  0 10]
 [ 8  4  3 27 19 21  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_256x256-df.csv


In [8]:
IMG_SIZE = 512

for attack_label in attack_labels:
    print(f'Type: {attack_label} \n')
    metrics = evaluate_model(
        model=model,
        checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
        detect_csv=DETECT_CSV,
        image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
        class_to_idx=class_to_idx,
        batch_size=2,        
        out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
        verbose=True
    )

Type: fgsm 

Loaded checkpoint '../01-CleanModel/Models/512x512/best_min_acc_vgg16_512x512_Model-2.pth' (strict=True).


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [01:43<00:00,  3.15it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 64  1  6 10  0  6]
 [15  0  2 13 32  0 12]
 [ 5  4  0 35  3  1  8]
 [16  7 73  0 11  3 44]
 [12 74  5  6  0  1 45]
 [ 0 27  2  3 12  0  9]
 [ 4 29  2 18 21  8  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_512x512-fgsm.csv
Type: bim 

Loaded checkpoint '../01-CleanModel/Models/512x512/best_min_acc_vgg16

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [01:42<00:00,  3.16it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 66  1  8  8  0  4]
 [12  0  2  9 35  1 15]
 [ 3  3  0 32  4  1 13]
 [15  6 72  0 12  4 45]
 [14 67  3  8  0  2 49]
 [ 0 26  2  2 13  0 10]
 [ 3 26  2 17 22 12  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_512x512-bim.csv
Type: pgd 

Loaded checkpoint '../01-CleanModel/Models/512x512/best_min_acc_vgg16_

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [01:50<00:00,  2.94it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 65  1  6 13  1  1]
 [19  0  2 10 33  1  9]
 [ 5  6  0 34  5  1  5]
 [12 10 79  0 13  2 38]
 [23 70  5  6  0  1 38]
 [ 1 25  2  5 11  0  9]
 [ 3 23  1 16 30  9  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_512x512-pgd.csv
Type: df 

Loaded checkpoint '../01-CleanModel/Models/512x512/best_min_acc_vgg16_5

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [01:41<00:00,  3.22it/s]


=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 48  4 15 14  1  5]
 [17  0  1 10 36  1  9]
 [ 4  5  0 40  2  1  4]
 [13 18 36  0 23  7 57]
 [25 40  3 27  0  5 43]
 [ 4  7  7 13  6  0 16]
 [ 8 15  1 29 19 10  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_512x512-df.csv


In [9]:
IMG_SIZE = 1024

for attack_label in attack_labels:
    print(f'Type: {attack_label} \n')
    metrics = evaluate_model(
        model=model,
        checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
        detect_csv=DETECT_CSV,
        image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
        class_to_idx=class_to_idx,
        batch_size=2,        
        out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
        verbose=True
    )

Type: fgsm 

Loaded checkpoint '../01-CleanModel/Models/1024x1024/best_min_acc_vgg16_1024x1024_Model-2.pth' (strict=True).


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [04:13<00:00,  1.28it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0  7  4 32 18 21  5]
 [11  0  9 33  9  3  9]
 [ 2  2  0 28 16  7  1]
 [ 1  3 72  0 42  3 33]
 [ 8 28  8 72  0 11 16]
 [11  9  0 11  9  0 13]
 [ 4  2  0 15 52  9  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_1024x1024-fgsm.csv
Type: bim 

Loaded checkpoint '../01-CleanModel/Models/1024x1024/best_min_acc_v

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [03:24<00:00,  1.59it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0  0  2 50  6 25  4]
 [ 4  0  2 67  0  1  0]
 [ 1  0  0 33 13  6  3]
 [ 1  1 72  0 38  8 34]
 [ 9  1  7 70  0 40 16]
 [19  0  0 29  0  0  5]
 [ 3  3  1 19 46 10  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_1024x1024-bim.csv
Type: pgd 

Loaded checkpoint '../01-CleanModel/Models/1024x1024/best_min_acc_vg

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [03:52<00:00,  1.40it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0  0  1 15 26 41  4]
 [11  0  0 61  0  2  0]
 [ 4  0  0 10 32  9  1]
 [ 6  4 47  0 61  4 32]
 [10  2 19 14  0 65 33]
 [26  0  0 25  0  0  2]
 [ 5 12  0  2 58  5  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_1024x1024-pgd.csv
Type: df 

Loaded checkpoint '../01-CleanModel/Models/1024x1024/best_min_acc_vgg

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [03:47<00:00,  1.43it/s]


=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 65  5  9  3  0  5]
 [33  0  4  5 17 15  0]
 [ 8  3  0 32  0  9  4]
 [14  5 75  0 27  9 24]
 [10 66  6 32  0 10 19]
 [ 0 23 14  7  2  0  7]
 [ 6 13  2  9 49  3  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_1024x1024-df.csv


In [13]:
IMG_SIZE = 32
attack_label = 'cw'
metrics = evaluate_model(
    model=model,
    checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
    detect_csv=DETECT_CSV,
    image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
    class_to_idx=class_to_idx,
    batch_size=2,        
    out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
    verbose=True
)

Loaded checkpoint '../01-CleanModel/Models/32x32/best_min_acc_vgg16_32x32_Model-2.pth' (strict=True).


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [00:55<00:00,  5.84it/s]


=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 54  6 17  3  1  6]
 [18  0  4 16 21  4 11]
 [ 9  3  0 37  2  3  2]
 [11 32 35  0 36  4 36]
 [10 38  1 31  0  1 62]
 [ 6  2 19 10  3  0 13]
 [ 4 10  0 42 15 11  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_32x32-cw.csv


In [14]:
IMG_SIZE = 64
attack_label = 'cw'
metrics = evaluate_model(
    model=model,
    checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
    detect_csv=DETECT_CSV,
    image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
    class_to_idx=class_to_idx,
    batch_size=2,        
    out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
    verbose=True
)

Loaded checkpoint '../01-CleanModel/Models/64x64/best_min_acc_vgg16_64x64_Model-2.pth' (strict=True).


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [01:03<00:00,  5.13it/s]


=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 51  9 18  7  1  1]
 [30  0  3  7 24  1  9]
 [ 3 13  0 37  0  3  0]
 [15 18 54  0 33  4 30]
 [14 35  4 33  0  5 52]
 [ 4  3  9 12  4  0 21]
 [ 6  8  0 29 25 14  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_64x64-cw.csv


In [15]:
IMG_SIZE = 128
attack_label = 'cw'
metrics = evaluate_model(
    model=model,
    checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
    detect_csv=DETECT_CSV,
    image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
    class_to_idx=class_to_idx,
    batch_size=2,        
    out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
    verbose=True
)

Loaded checkpoint '../01-CleanModel/Models/128x128/best_min_acc_vgg16_128x128_Model-2.pth' (strict=True).


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [01:42<00:00,  3.17it/s]


=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 53  5 17 10  1  1]
 [26  0  2  4 35  1  6]
 [ 2  6  0 46  1  0  1]
 [10 26 39  0 31  6 42]
 [ 8 55  1 16  0  5 58]
 [ 9  8  8  4  7  0 17]
 [ 8  6  3 29 26 10  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_128x128-cw.csv


In [16]:
IMG_SIZE = 256
attack_label = 'cw'
metrics = evaluate_model(
    model=model,
    checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
    detect_csv=DETECT_CSV,
    image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
    class_to_idx=class_to_idx,
    batch_size=2,        
    out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
    verbose=True
)

Loaded checkpoint '../01-CleanModel/Models/256x256/best_min_acc_vgg16_256x256_Model-2.pth' (strict=True).


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 325/325 [00:57<00:00,  5.64it/s]


=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 52  5 13 10  1  6]
 [18  0  2  9 39  2  4]
 [ 4  5  0 44  0  1  2]
 [ 8 14 50  0  9  6 67]
 [14 46  3 22  0  3 55]
 [ 8 11  3 11  4  0 16]
 [ 7  6  2 30 16 21  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_256x256-cw.csv


In [4]:
IMG_SIZE = 512
attack_label = 'cw'
metrics = evaluate_model(
    model=model,
    checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
    detect_csv=DETECT_CSV,
    image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
    class_to_idx=class_to_idx,
    batch_size=1,        
    out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
    verbose=True
)

Loaded checkpoint '../01-CleanModel/Models/512x512/best_min_acc_vgg16_512x512_Model-2.pth' (strict=True).


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 649/649 [01:03<00:00, 10.23it/s]



=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 55  3 11 12  1  5]
 [18  0  1 10 36  1  8]
 [ 3  5  0 41  3  0  4]
 [13 12 63  0  9  8 49]
 [13 58  3 20  0  4 45]
 [ 2 20  1 10  9  0 11]
 [ 3 21  1 23 24 10  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_512x512-cw.csv


In [5]:
IMG_SIZE = 1024
attack_label = 'cw'
metrics = evaluate_model(
    model=model,
    checkpoint_path=f'../01-CleanModel/Models/{IMG_SIZE}x{IMG_SIZE}/best_min_acc_vgg16_{IMG_SIZE}x{IMG_SIZE}_Model-2.pth',
    detect_csv=DETECT_CSV,
    image_dir=f"./generated_images-test/{IMG_SIZE}x{IMG_SIZE}/{attack_label}/",
    class_to_idx=class_to_idx,
    batch_size=1,        
    out_csv_path=Path(f"./Evaluate/raw/miss-detection_results_{IMG_SIZE}x{IMG_SIZE}-{attack_label}.csv"),
    verbose=True
)

Loaded checkpoint '../01-CleanModel/Models/1024x1024/best_min_acc_vgg16_1024x1024_Model-2.pth' (strict=True).


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 649/649 [02:21<00:00,  4.59it/s]


=== Classification Report ===
              precision    recall  f1-score   support

        bird       0.00      0.00      0.00      87.0
         cat       0.00      0.00      0.00      74.0
        fish       0.00      0.00      0.00      56.0
        frog       0.00      0.00      0.00     154.0
      monkey       0.00      0.00      0.00     143.0
        moth       0.00      0.00      0.00      53.0
      spider       0.00      0.00      0.00      82.0

    accuracy                           0.00     649.0
   macro avg       0.00      0.00      0.00     649.0
weighted avg       0.00      0.00      0.00     649.0

=== Confusion Matrix ===
[[ 0 61 10 10  2  0  4]
 [24  0  3  4 33 10  0]
 [ 8  3  0 30  1 11  3]
 [ 4  4 63  0 40 10 33]
 [ 6 51  5 39  0  1 41]
 [ 0 32  3  9  0  0  9]
 [ 6  9  2 13 48  4  0]]

Macro F1: 0.0000
Saved detailed results to Evaluate\raw\miss-detection_results_1024x1024-cw.csv
